Transformers for Sentiment Analysis
====

In questo notebook andremo ad usare i transformers, sono stati introdotti per la prima volta in [questo paper](https://arxiv.org/abs/1706.03762). Nello specifico andremo ad utilizzare BERT (Bidirectional Encoder Representations from Transformers) descritto in [questo paper](https://arxiv.org/abs/1810.04805).

I transformer sono modelli considerevolmente più grandi dei precedenti, andremo ad utilizzare una libreria che ci permette di avvicinarci a questi tipo di architetture senza grosse difficoltà. La [libreria](https://github.com/huggingface/transformers) ci permette di scaricare un modello pre-trained, congeleremo il transformer e addestreremo solo il modello che impara dal risultato del transformer.

Preparazione dei dati
----

Per prima cosa impostiamo il seme dei numeri casuali per ottenere un risultato uguale.

In [1]:
import torch

import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Il transformer è stato addestrato su un vocabolario specifico, il che significa che dobbiamo replicare lo stesso vocabolario nel nostro datase e dobbiamo creare i token nello stesso modo in cui il trasformer è stato inizialmente addestrato.

Fortunatamente la libreria transformers ha un tokenizer per ogni modello di transformer fornito. In questo caso useremo BERT ignorando le parole maiuscole (metteremo tutto in minuscolo). Lo facciamo caricando il tokenizer bert-base-uncased.

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

INFO:transformers.file_utils:PyTorch version 1.5.0 available.
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at C:\Users\fabio\.cache\torch\transformers\26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


Il `tokenizer` ha un attributo `vocab` che contiene il vocabolario che verrà utilizzato. Possiamo controllare il numero di vocaboli utilizzati andando a controllare la sua lunghezza.

In [4]:
len(tokenizer.vocab)

30522

Per utilizzare il `tokenizer` basta semplicemente chiamare `tokenizer.tokenize` sulla stringa. Questo andrà a tokenizzare e mettere in minuscolo la frase nella maniera più efficente per il modello pre-trained scelto.

In [5]:
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')

print(tokens)

['hello', 'world', 'how', 'are', 'you', '?']


possiamo trasformare in numeri i token appena generati

In [6]:
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

[7592, 2088, 2129, 2024, 2017, 1029]


Il transformer è stati addestrato con dei token speciali per segnare l'inizio e la fine di una frase.
Abbiamo anche i token per la gestione del padding e delle parole sconosciute. Possiamo ottenere gli indici dal tokenizer..

In [7]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


possiamo ottenere gli indici di questi token speciali convertendoli usando il dizionario

In [8]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


o andandoli a leggere direttamente dal tokenizer

In [9]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


Un'altra cosa da gestire è che il modello è stato addestrato su delle sequenze con una lunghezza massima predefinita. Perciò non possiamo dargli in pasto sequenze maggiori con cui è stato addestrato. Possiamo ottenere la lunghezza massima di questo input andando a controllare `max_model_input_sizes` per la versione del trasformer che vogliamo usare. Nel nostro caso 512 token.

In [10]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']
print(max_input_length)

512


Precedentemente abbiamo usato spaCy come tokenizer. Dobbiamo impostare una funzione a cui passeremo il nostro campo TEXT e che eseguirà il lavoro per noi. 
Dobbiamo anche tagliare il numero di token alla massima lunghezza consentita.

Nota che la nostra massima lunghezza è ridotta di due unità in quanto dobbiamo inserire un token all'inizio e alla fine di ogni frase.

In [11]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

Ora possiamo definire i nostri field. Il tranformer si aspetta come prima dimensione quella del batch, dunque impostiamo `batch_first = True`. Abbiamo già un vocabolario per il nostro testo, dato dal vocabolario del transformer dunque impostiamo `use_vocab = False` per dire a torchtext che useremo un nostro vocabolario.

Impostiamo la nostra funzione `tokenize_and_cut` come tokenizer. Il parametro `preprocessing` è una funzione che viene invocata dopo il tokenize. Utilizzeremo questo punto per convertire i nostri token in indici.

Alla fine impostiamo i special tokens, facendo attenzione a definirli come indici e non come stringhe questo perchè la stringa è già stata convertita in indici. 

Impostiamo anche il campo label come già fatto in precedenza.

In [12]:
from torchtext import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

Come al solito creiamo il dataset di validazione

In [13]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [14]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


Andiamo ad analizzare un record per controllare il risultato della elaborazione

In [15]:
print(vars(train_data.examples[6]))

{'text': [1996, 18458, 1997, 6644, 9016, 4627, 2066, 2009, 2453, 2031, 2242, 2000, 3749, 1012, 1037, 2177, 1997, 2267, 13496, 2044, 4399, 1006, 1999, 1996, 2991, 1029, 1007, 3632, 2000, 1037, 7001, 6644, 1999, 1996, 5249, 2073, 2028, 2011, 2028, 2027, 2024, 4457, 2011, 2019, 16100, 5771, 5983, 7865, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 6854, 1010, 1996, 2034, 20423, 2003, 2073, 2151, 6556, 3787, 1997, 2143, 3737, 2644, 1012, 6644, 9016, 2003, 2210, 2062, 2084, 2267, 4268, 2559, 2005, 3348, 1010, 22017, 4371, 1010, 3331, 2512, 1011, 2644, 2055, 2498, 1010, 1998, 3773, 2129, 2116, 1042, 1011, 9767, 2027, 2064, 2131, 2046, 1015, 1024, 2871, 2781, 2030, 2174, 2146, 2023, 6752, 2003, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 4268, 2552, 1998, 10509, 5236, 2135, 2000, 2673, 2105, 2068, 1012, 2028, 1997, 2068, 2005, 6013, 9418, 2008, 1996, 3096, 7865, 2038, 10372, 2014, 3456, 1010, 2061, 2054, 2515, 2016, 2079, 1029, 2016, 7906, 21146, 6455, 2014, 3456, 7989

Possiamo utilizzare la funzione `convert_ids_to_tokens` per trasformare gli indici in una lista di token leggibili. 

In [16]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])
print(tokens)

['the', 'premise', 'of', 'cabin', 'fever', 'starts', 'like', 'it', 'might', 'have', 'something', 'to', 'offer', '.', 'a', 'group', 'of', 'college', 'teens', 'after', 'finals', '(', 'in', 'the', 'fall', '?', ')', 'goes', 'to', 'a', 'resort', 'cabin', 'in', 'the', 'woods', 'where', 'one', 'by', 'one', 'they', 'are', 'attacked', 'by', 'an', 'unseen', 'flesh', 'eating', 'virus', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'unfortunately', ',', 'the', 'first', 'paragraph', 'is', 'where', 'any', 'remote', 'elements', 'of', 'film', 'quality', 'stop', '.', 'cabin', 'fever', 'is', 'little', 'more', 'than', 'college', 'kids', 'looking', 'for', 'sex', ',', 'boo', '##ze', ',', 'talking', 'non', '-', 'stop', 'about', 'nothing', ',', 'and', 'seeing', 'how', 'many', 'f', '-', 'bombs', 'they', 'can', 'get', 'into', '1', ':', '40', 'minutes', 'or', 'however', 'long', 'this', 'mess', 'is', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'the', 'kids', 'act', 'and', 'react', 'stupid', '##ly', 'to', 'ev

Sebbene abbiamo già un vocabolario per il testi, dobbiamo comunque creare un vocabolario per le label.

In [17]:
LABEL.build_vocab(train_data)
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


Come prima creaiamo gli iteratori. Cerchiamo di utilizzare la dimensione del batch size più grande possibile per ottenere il miglior risultato con i transformer. 

In [18]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

Costruiamo il modello
----

Ora andremo a caricare il pre-trained model, facendo attenzione nel caricare lo stesso modello usato nel tokenizer.

In [19]:
from transformers import BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at C:\Users\fabio\.cache\torch\transformers\4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
INFO:transformers.configuration_utils:Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention

Ora impostiamo il nostro modello.

Invece di utilizzare un embedding layer per fare il lavoro, andremo ad utilizzare il risultato del nostro pre-trained transformer.

Questo embedding verrà utilizzato per alimentare una rete neuronale la quale ci darà il nostro risultato. Possiamo ottenere la dimensione di embedding chiamata anche `hidden_size` dal transformer dal suo parametro config.

Il resto della inizializzazione è standard.

Nel passo di forward, andiamo a impostare il parametro no_grad per essere sicuri che il nostro gradiente sia calcolato fuori dal modello bert.

Il transfomer ritorna un embedding per l'intera sequenza cosi come un pooled output.
Il pooled output è una sorta di riassunto del contenuto anche se la documentazione riporta che: "spesso non è un buon riassunto del contenuto semantico dell'input è meglio prendere la sequenza di hidden state e fare averaging o pooling della sequenza di hidden state".

In [20]:
print(bert.config.hidden_size)

768


Andiamo ad analizzare le dimensioni dell'output del modello bert

In [21]:
import torch

sequence = "This film wasn't great"

input_ids = torch.tensor(tokenizer.encode(sequence, add_special_tokens=True)).unsqueeze(0)  # Batch size 1

print(input_ids.shape)
tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze(0))
print(tokens)

outputs = bert(input_ids)

last_hidden_states = outputs[0]
print(last_hidden_states.shape)


pooler_output = outputs[1]
print(pooler_output.shape)


torch.Size([1, 8])
['[CLS]', 'this', 'film', 'wasn', "'", 't', 'great', '[SEP]']
torch.Size([1, 8, 768])
torch.Size([1, 768])


Come scritto nella documentazione andiamo a provare il codice per l'averaging (media) dei tensori dell'hidden state

In [22]:
import torch.nn.functional as F
pooled = F.avg_pool2d(last_hidden_states, (last_hidden_states.shape[1], 1)).squeeze(1) 
print(pooled.shape)

torch.Size([1, 768])


Possiamo utilizzare anche un modello per la classificazione completo [BertForSequenceClassification](https://huggingface.co/transformers/_modules/transformers/modeling_bert.html#BertForSequenceClassification) ma preferisco creare un modello da zero per capire il più possibile il funzionamento di bert. Qui sotto comunque un esempio di funzionamento 

In [23]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

#configurazione
model.config

input_ids = torch.tensor(tokenizer.encode(sequence, add_special_tokens=True)).unsqueeze(0)
outputs = model(input_ids)[0]
print(outputs)
values, indices = torch.max(outputs,1)
print(indices,values)


INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at C:\Users\fabio\.cache\torch\transformers\4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
INFO:transformers.configuration_utils:Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention

tensor([[0.0765, 0.3524]], grad_fn=<AddmmBackward>)
tensor([1]) tensor([0.3524], grad_fn=<MaxBackward0>)


Andiamo a creare il modello e instanziarlo

In [24]:
import torch.nn as nn

class BERTSentiment(nn.Module):
    def __init__(self,bert,output_dim,dropout):
        
        super().__init__()
        
        self.bert = bert
        embedding_dim = bert.config.to_dict()['hidden_size']
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(embedding_dim, output_dim)
        
       
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
           last_hidden_states,_ = self.bert(text)
        
        pooled = F.avg_pool2d(last_hidden_states, (last_hidden_states.shape[1], 1)).squeeze(1) 
        #pooled = [batch size, emb dim]
        
        pooled = self.dropout(pooled)
        output = self.out(pooled)
        
        #output = [batch size, out dim]
        
        return output

In [25]:
OUTPUT_DIM = 1
DROPOUT = 0.25
model = BERTSentiment(bert,OUTPUT_DIM,DROPOUT)

Andiamo a controllare il numero dei parametri del modello.
Fino ad ora non abbiamo superato 5M, ma questo modello supera i 112 Milioni di parametri. Fortunatamente ben 100 Milioni di parametri sono del transformer e non devono essere usati nella fase di train.

In [26]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 109,483,009 trainable parameters


Per bloccare i parametri dobbiamo impostare il parametro `requires_grad` a False. Per fare questo, semplicemente controlliamo tutti i `named_parameters` nel nostro modello e se fanno parte del nostro transformer impostiamo `requires_grad = False`.

In [27]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [28]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 769 trainable parameters


A solo scopo didattico andiamo a vedere quali sono i parametri che possono essere usati per addestrare il modello

In [29]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

out.weight
out.bias


Train del modello
----

Definiamo l'ottimizzatore e la funzione di loss

In [30]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

In [31]:
model = model.to(device)
criterion = criterion.to(device)

Usiamo la nostra consueta funzione di accuracy e impostiamo il codice per il train/evaluation.

In [32]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [33]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [34]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [35]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Finalmente siamo pronti per la fase di train. Come vedremo ora i tempi sono molto più lunghi dei precedenti modelli, questo è dovuto alla dimensione del transformer.

Anche se non stiamo eseguendo nessun calcolo del gradiente con i parametri del transformer dobbiamo sempre fare tutti i calcoli per ottenere il vettore finale. Questo consuma un bel po di tempo anche in una GPU.

In [36]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 11m 35s
	Train Loss: 0.651 | Train Acc: 64.14%
	 Val. Loss: 0.533 |  Val. Acc: 79.47%
Epoch: 02 | Epoch Time: 11m 3s
	Train Loss: 0.590 | Train Acc: 72.68%
	 Val. Loss: 0.464 |  Val. Acc: 80.93%
Epoch: 03 | Epoch Time: 11m 5s
	Train Loss: 0.558 | Train Acc: 74.43%
	 Val. Loss: 0.427 |  Val. Acc: 81.93%


KeyboardInterrupt: 

Una volta eseguito il train andiamo a caricare i parametri e a vedere il comportamento del modello sul dataset di test

In [37]:
model.load_state_dict(torch.load('tut5-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.424 | Test Acc: 81.80%


Input personalizzato
---

Controlliamo il modello per verificare alcune sequenze. Andremo a creare i token della sequenza di input, la taglieremo alla massima lunghezza impostando i token speciali. Come ultima cosa si crea il tensore per create un finto batch da dare al nostro modello.

In [38]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [39]:
predict_sentiment(model, tokenizer, "This film is terrible")

0.11642389744520187

In [40]:
predict_sentiment(model, tokenizer, "This film is great")

0.8123869895935059